<a href="https://colab.research.google.com/github/alhe/Bias_mitigation/blob/main/UPP26/TD2_mepsdata_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MiProjet Bernabé Clarence



## Introduction
Pour ce projet, nous allons analyser un sous-ensemble des métasdonnées de 15000 individus, issus du jeu de données Chest X ray NIH 14 Dataset : https://www.kaggle.com/datasets/nih-chest-xrays/data. Le fichier contenant les données de départs est Bernabe_Clarence.csv.

### Objectifs:
1. Analyse des métadonnées
2. Identification de biais (s'il en existe)
3. Réduction des biais grâce à une méthode de pré-processing (nouveau dataset avec biais réduits)

### Présentation du dataset
Ce dataset contient 53467 lignes, chacune avec 11 features qui sont : Image Index, Finding Labels, Follow-up #, Patient ID, Patient Age, Patient Gender, View Position, OriginalImage[Width, Heigth], OriginalImagePixelSpacing[x, y].  
Chaque ligne correspond à un scanner différent, et ces scanners ont été effectués sur 15000 patients (Donc plusieurs scanners ont été effectués sur certains patients).  
La colonne Findings Labels correspond à la maladie dont souffre le patient.  
Follow-up # indique le rang chronologique de la radiographie pour un patient (0 = première visite, 1 = deuxième visite , ect...).  
View Position correspond à l'angle de pris de vue, PA (Postero-Anterior, debout) ou AP (Antero-Posterior, allongé).  


In [10]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.8/191.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

ImportError: cannot import name 'xp_broadcast_promote' from 'scipy._lib._array_api' (/usr/local/lib/python3.12/dist-packages/scipy/_lib/_array_api.py)

In [14]:
import numpy as np
import fairlearn
import plotly.express as px
import pandas as pd
#print(np.__version__,fairlearn.__version__)
print("Importation des bibliothèques réussi.\n")

Importation des bibliothèques réussi.



In [15]:
url = "https://raw.githubusercontent.com/ClarenceBrn/Fairness/main/Bernabe_Clarence.csv"
df = pd.read_csv(url)
print("Chargement du dataset sous forme dataframe réussi.\n")

Chargement du dataset sous forme dataframe réussi.



In [47]:
df


,Image_Index,Finding_Labels,Follow-up_#,Patient_ID,Patient_Age,Patient_Gender,View_Position,OriginalImWidth,OriginalImHeight,OriginalImPixSpacingX,OriginalImPixSpacingY
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143000,0.143000
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143000,0.143000
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168000,0.168000
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171000,0.171000
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143000,0.143000
...,...,...,...,...,...,...,...,...,...,...,...
53462,00030787_000.png,No Finding,0,30787,34,M,PA,2021,2021,0.194311,0.194311
53463,00030789_000.png,Infiltration,0,30789,52,F,PA,2021,2021,0.194311,0.194311
53464,00030797_000.png,No Finding,0,30797,24,M,PA,2021,2021,0.194311,0.194311
53465,00030798_000.png,No Finding,0,30798,30,M,PA,2500,2048,0.171000,0.171000


#### Modification du noms features pour faciliter la manipulation des données
Les nouveaux noms sont : ['Image_Index', 'Finding_Labels', 'Follow-up_#', 'Patient_ID',
       'Patient_Age', 'Patient_Gender', 'View_Position', 'OriginalImageWidth',
       'OriginalImageHeight', 'OriginalImagePixSpacingX', 'OriginalImagePixSpacingY'].

In [29]:
df = df.rename(columns={
    'OriginalImage[Width' : 'OriginalImageWidth',
    'Height]' : 'OriginalImageHeight',
    'OriginalImagePixelSpacing[x' : 'OriginalImagePixSpacingX',
    'y]' : 'OriginalImagePixSpacingY'
})
df.columns = df.columns.str.replace(' ', '_')
df.columns


Index(['Image_Index', 'Finding_Labels', 'Follow-up_#', 'Patient_ID',
       'Patient_Age', 'Patient_Gender', 'View_Position', 'OriginalImWidth',
       'OriginalImHeight', 'OriginalImPixSpacingX', 'OriginalImPixSpacingY'],
      dtype='object')

## Analyse des données

In [32]:
# On vérifie qu'il n'y ai pas de données manquantes
for column in df.columns:
  print("Données manquantes pour la feature ",column, " = ", df[column].isnull().values.any())

Données manquantes pour la feature  Image_Index  =  False
Données manquantes pour la feature  Finding_Labels  =  False
Données manquantes pour la feature  Follow-up_#  =  False
Données manquantes pour la feature  Patient_ID  =  False
Données manquantes pour la feature  Patient_Age  =  False
Données manquantes pour la feature  Patient_Gender  =  False
Données manquantes pour la feature  View_Position  =  False
Données manquantes pour la feature  OriginalImWidth  =  False
Données manquantes pour la feature  OriginalImHeight  =  False
Données manquantes pour la feature  OriginalImPixSpacingX  =  False
Données manquantes pour la feature  OriginalImPixSpacingY  =  False


In [33]:
print("Nombre de patient différent : " + str(df["Patient_ID"].nunique())+"\n")
print("Dimension du dataset : " + str(df.shape) + "\n")
print("Nombre de labels différent : " + str(df["Finding_Labels"].nunique()) + "\n")
print("Tranche d'âge des patients : " + str(df["Patient_Age"].min()) + " -> " + str(df["Patient_Age"].max()) + "\n")
print("Les différents genres de patients : " + str(df["Patient_Gender"].unique()) + "\n")
print("Les différents angles de prise de vue : " + str(df["View_Position"].unique()) + "\n")

Nombre de patient différent : 15000

Dimension du dataset : (53467, 11)

Nombre de labels différent : 603

Tranche d'âge des patients : 1 -> 414

Les différents genres de patients : ['M' 'F']

Les différents angles de prise de vue : ['PA' 'AP']



On remarque que les patients ont entre 1 et 414 ans. Cette borne supérieur est absurde, essayons de la corriger.

In [43]:
df[df.Patient_Age > 120]

,Image_Index,Finding_Labels,Follow-up_#,Patient_ID,Patient_Age,Patient_Gender,View_Position,OriginalImWidth,OriginalImHeight,OriginalImPixSpacingX,OriginalImPixSpacingY
9834,00005567_000.png,Effusion|Pneumonia,0,5567,412,M,AP,3056,2544,0.139,0.139
22095,00011973_002.png,Edema,2,11973,414,M,AP,3056,2544,0.139,0.139
22866,00012238_010.png,No Finding,10,12238,148,M,PA,2992,2991,0.143,0.143
30265,00015558_000.png,No Finding,0,15558,149,M,PA,2992,2991,0.143,0.143
35703,00018366_044.png,Pneumothorax,44,18366,152,F,PA,2302,2991,0.143,0.143
37603,00019346_000.png,Infiltration,0,19346,151,F,PA,2678,2774,0.143,0.143
40820,00021047_002.png,Mass|Pleural_Thickening,2,21047,412,M,AP,3056,2544,0.139,0.139
41197,00021275_003.png,No Finding,3,21275,413,F,AP,3056,2544,0.139,0.139
43627,00022811_000.png,No Finding,0,22811,412,M,PA,3056,2544,0.139,0.139
45685,00025206_000.png,Infiltration|Mass,0,25206,153,M,PA,2992,2991,0.143,0.143


Cherchons maintenant si les patients avec ces âges ont d'autres scanners avec cette fois un âge plus cohérent grâce à leurs Patient_Id.

In [49]:
ids_absurdes = df[df.Patient_Age > 120].Patient_ID.unique()
lignes_ids_absurdes = df[df.Patient_ID.isin(ids_absurdes)]
lignes_triees = lignes_ids_absurdes.sort_values(by=['Patient_ID', 'Follow-up_#'])
historique_ages = lignes_triees.groupby('Patient_ID')['Patient_Age'].apply(list)
#for id_patient, liste_ages in historique_ages.items():
    #print(f"Patient {id_patient} : {liste_ages}")


Patient 5567 : [412, 53, 53, 53, 54, 54, 54, 57, 57, 57, 44, 47, 48, 48, 48, 49, 49, 50, 50, 51, 51, 51, 51, 52, 53, 53]
Patient 11973 : [65, 65, 414, 58, 58, 58, 59, 59, 60, 61, 60, 61, 61, 61, 61, 62, 62, 62, 62, 62, 62, 63]
Patient 12238 : [63, 63, 63, 63, 63, 63, 64, 63, 63, 64, 148, 64, 64, 64, 64]
Patient 15558 : [149, 46, 46, 46]
Patient 18366 : [64, 65, 64, 64, 64, 64, 64, 65, 64, 64, 65, 64, 64, 64, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 65, 64, 64, 64, 64, 152, 64, 64, 64, 64, 64, 64, 64, 64, 65, 65, 65, 65, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66]
Patient 19346 : [151]
Patient 21047 : [52, 52, 412, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52]
Patient 21275 : [21, 21, 21, 413, 22, 19, 19, 19]
Patient 22811 : [412, 25, 25, 25, 25, 25]
Patient 25206 : [153, 36, 36, 36, 36]
Patient 26028 : [60, 154]
Patient 26871 : [155]
Patient 27989 : [155]


On va décider de supprimer les lignes avec un âge absurde lorsque les patients ne sont venus qu'une fois (seulement 3 lignes concernées), et sinon de modifier la valeur lorsqu'elle est absurde grâce aux valeurs des autres visites.

In [52]:
corrections = {
    5567: 53,
    11973: 61,
    12238: 64,
    15558: 46,
    18366: 64,
    21047: 52,
    21275: 21,
    22811: 25,
    25206: 36,
    26028: 60
}

for patient_id, new_age in corrections.items():
    df.loc[(df['Patient_ID'] == patient_id) & (df['Patient_Age'] > 100), 'Patient_Age'] = new_age

df = df[df['Patient_Age'] <= 100]
print("Tranche d'âge des patients : " + str(df["Patient_Age"].min()) + " -> " + str(df["Patient_Age"].max()) + "\n")

Tranche d'âge des patients : 1 -> 94



On a réussi à gérer les valeurs absurdes présentes dans le jeu de données.